In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# URL
url = "https://en.wikipedia.org/wiki/1936_United_States_presidential_election"

# Request
response = requests.get(url)
soup = BeautifulSoup(response.content)

response.status_code

In [ ]:
# Get only target table
target_table = None
for table in soup.find_all("table", class_="wikitable"):
    caption = table.find("caption")
    if caption and "Electoral results" in caption.text: #from inspect on site
        target_table = table
        

# Read table into pandas df
df_1_actual_results = pd.read_html(str(target_table))[0]


df_1_actual_results.head(25)

In [ ]:
# Get rid of multi index column headers

# Flatten multi index column headers
df_1_actual_results.columns = ['_'.join(col).strip().replace(" ", "_").lower() for col in df_1_actual_results.columns.values]

df_1_actual_results.head()

In [ ]:
# Add year column for year 1936 for all rows in df

df_1_actual_results['year'] = 1936

df_1_actual_results.head(25)

In [ ]:
df_1_actual_results.columns

In [ ]:
# drop presidential_candidate_presidential_candidate, home_state_home_state, running_mate_vice-presidential_candidate, 
#running_mate_home_state, running_mate_electoral_vote

columns_to_drop = [
    'presidential_candidate_presidential_candidate',
    'home_state_home_state',
    'running_mate_vice-presidential_candidate',
    'running_mate_home_state',
    'running_mate_electoral_vote'
]

df_1_actual_results = df_1_actual_results.drop(columns=columns_to_drop)


df_1_actual_results.head(25)

In [ ]:
# drop rows 2-7 (Union to Other)
df_1_actual_results = df_1_actual_results.drop(index=range(2, 8))

# Reset index
df_1_actual_results = df_1_actual_results.reset_index(drop=True)

df_1_actual_results.head(25)

In [ ]:
# Rename party_party to party
# Rename electoral_vote_electoral_vote to electoral_vote

df_1_actual_results = df_1_actual_results.rename(columns={
    'party_party': 'party',
    'electoral_vote_electoral_vote': 'electoral_vote'
})

df_1_actual_results.head()

In [ ]:
df_1_actual_results.columns

In [ ]:
df_filtered = df_1_actual_results[df_1_actual_results['party'] != 'Needed to win']

df_filtered.head()

In [ ]:
# Pivot
df_wide = df_filtered.set_index('party').T

df_wide.head()

In [ ]:
# Rename Total column to match 1936 table
df_wide = df_wide.rename(columns={'Total': 'Total_Popular_Vote'})

df_wide.head()

In [ ]:
# Putting everything we learned together and seeing if it works
# Match to 1936 table for easy merge

rep_electoral = int(df_wide.loc['electoral_vote', 'Republican'])
dem_electoral = int(df_wide.loc['electoral_vote', 'Democratic'])
total_electoral = int(df_wide.loc['electoral_vote', 'Total_Popular_Vote'])

rep_popular = int(df_wide.loc['popular_vote_count', 'Republican'])
dem_popular = int(df_wide.loc['popular_vote_count', 'Democratic'])
total_popular = int(df_wide.loc['popular_vote_count', 'Total_Popular_Vote'])

# turn into floats
rep_popular_pct = float(df_wide.loc['popular_vote_percentage', 'Republican'].replace('%',''))
dem_popular_pct = float(df_wide.loc['popular_vote_percentage', 'Democratic'].replace('%',''))

In [ ]:
# Calculate Electoral Percentages and Leading Margins

rep_electoral_pct = round(rep_electoral / total_electoral * 100, 2)
dem_electoral_pct = round(dem_electoral / total_electoral * 100, 2)

electoral_margin = abs(dem_electoral_pct - rep_electoral_pct)
popular_margin = abs(dem_popular_pct - rep_popular_pct)

In [ ]:
final_df = pd.DataFrame({
    'Republican_Electoral':[rep_electoral],
    
    'Democrat_Electoral':[dem_electoral],
    
    'Republican_Popular':[rep_popular],
    
    'Democrat_Popular':[dem_popular],
    
    'Total_Popular_Vote':[total_popular],

    'Republican_Electoral_pct':[rep_electoral_pct],
    
    'Democrat_Electoral_pct':[dem_electoral_pct],
    
    'Republican_Popular_pct':[rep_popular_pct],
    
    'Democrat_Popular_pct':[dem_popular_pct],

    'Electoral_Leading_Margin':[electoral_margin],
    
    'Popular_Leading_Margin':[popular_margin]
})

final_df.head()

In [ ]:
final_df.to_csv('1936_Actual_Election_Results.csv', index=False)

In [6]:
# Combine both datasets

poll_df = pd.read_csv("1936_Opinion_Polling_Data.csv")
actual_df = pd.read_csv("1936_Actual_Election_Results.csv")

FileNotFoundError: [Errno 2] No such file or directory: '1936_Opinion_Polling_Data.csv'

In [ ]:
# Confirm shapes 
poll_df.shape
actual_df.shape

In [ ]:
# Combine horizontally where axis=1
combined_df = pd.concat([poll_df, actual_df], axis=1)

combined_df.head(25)

In [ ]:
# Fill all NaN from Actual Election Results with value for all rows
# For example, the whole of Republican_Electoral column should be 8.0
actual_cols = [
    'Republican_Electoral','Democrat_Electoral'
    'Republican_Popular', 'Democrat_Popular',
    'Total_Popular_Vote', 'Republican_Electoral_pct',
    'Democrat_Electoral_pct', 'Republican_Popular_pct',
    'Democrat_Popular_pct', 'Electoral_Leading_Margin',
    'Popular_Leading_Margin'
]

In [ ]:
for col in actual_cols:
    combined_df[col] = combined_df[col].iloc[0]

combined_df.head(25)

In [ ]:
# add columns Poll_vs_Electoral_Margin_Diff	(poll leading - electoral leading)
# and Poll_vs_Popular_Margin_Diff (poll leading - popular leading)

combined_df['Poll_vs_Electoral_Margin_Diff'] = combined_df['Poll_Leading_Margin'] - combined_df['Electoral_Leading_Margin']
combined_df['Poll_vs_Popular_Margin_Diff'] = combined_df['Poll_Leading_Margin'] - combined_df['Popular_Leading_Margin']

combined_df.head(25)

In [ ]:
combined_df.to_csv("1936_Poll_v_Actual_dataset_with_diffs.csv", index=False)

In [ ]:
combined_df.columns